# 구글포토 이미지 OCR 적용후 문제유형
1. 구글포토 앨범이름으로 이미지 읽기
2. 구글 vision API 로 이미지 OCR 적용
3. openAI API로 문제, 보기 유형으로 변환
4. 변환된 파일로 apkg 생성


# 구글 vision API 인증키 json 파일 다운로드
1. google vision API 사용설정
구글클라우드 콘솔에서 vision API 사용설정후 인증키 json 파일 다운로드
현재 client_secret_desktop.json 파일로 사용
2. google photo 사용설정
구글포토 인증키 service-account-file.json 파일로 사용

In [1]:
%pip install --upgrade google-api-python-client
%pip install --upgrade google-cloud-vision
%pip install --upgrade google-auth-oauthlib

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviou

### 기출이라는 앨범에서 이미지 OCR 적용후 문제유형으로 변환


In [6]:
import html
import json
import os
import requests
from io import BytesIO
from google.cloud import vision
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import genanki
from openai import OpenAI

# OpenAI 클라이언트 설정
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)
open_ai_model = 'gpt-3.5-turbo'

# OAuth 2.0 클라이언트 ID와 비밀 정보 설정
client_secrets_file = 'client_secret_desktop.json'

# 스코프 설정
scopes = ['https://www.googleapis.com/auth/photoslibrary.readonly', 'https://www.googleapis.com/auth/cloud-vision']

# 사용자 인증 흐름 설정
flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes=scopes)

# 사용자 인증 및 액세스 토큰 획득
credentials = flow.run_local_server(port=8088)

# Google Photos API 서비스 객체 생성
service = build('photoslibrary', 'v1', credentials=credentials, static_discovery=False)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'service-account-file.json'
# Google Cloud Vision API 서비스 객체 생성
client_vision = vision.ImageAnnotatorClient(client_options={'api_endpoint': 'us-vision.googleapis.com'})

# 정답 리스트
answer_list = [
    '①', '④', '②', '③', '③',
    '①', '②', '④', '②', '①',
    '④', '③', '③', '③', '③',
    '②', '①', '④', '④', '②',
    '②', '③', '③', '③', '②',
    '①', '③', '①', '②', '③',
    '①', '③', '③', '②', '①',
    '①', '④', '①', '③', '②',
    '③', '④', '③', '②', '①',
    '①', '②', '①', '③', '①',
    '①', '②', '③', '①', '③',
    '①', '①', '②', '③', '②',
    '①', '④', '③', '②', '②',
    '②', '④', '②', '③', '④',
    '①', '①', '②', '③', '①',
    '④', '④', '②', '②', '①'
]


def get_photo_urls(examination):
    """앨범에서 사진의 URL 가져오는 함수"""
    # us-vision.googleapis.com

    response = service.mediaItems().search(body={'albumId': examination}).execute()
    items = response.get('mediaItems', [])
    return [item['baseUrl'] for item in items]


def annotate_image(image_url):
    """이미지를 Vision API 사용하여 주석  처리하는 함수"""
    # 이미지 너비를 원본크기로 설정하고, 높이를 원본 크기로 설정하며, 이미지를 잘라내지 않고 압축하지 않으며 오버레이를 표시하지 않습니다.
    original_image = '=w0-h0-n-k-no'
    response = requests.get(image_url + original_image, stream=True)
    if response.status_code == 200:
        image_bytes = BytesIO(response.content)
        image_byte_array = image_bytes.getvalue()
        content = {'content': image_byte_array}
        image = vision.Image(**content)
        # image.show()
        features = [{"type_": vision.Feature.Type.DOCUMENT_TEXT_DETECTION}]
        return client_vision.annotate_image({"image": image, "features": features}).full_text_annotation


def convert_ocr_to_text(document):
    text = ""

    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    text += "".join([symbol.text for symbol in word.symbols])
                    text += " "
                text += "\n"
    return text.strip()


# OCR 파이프라인 실행 함수
def run_ocr_pipeline(album_title, deck_title, apkg_title):
    response = service.albums().list().execute()

    # 앨범 목록 가져오기
    albums = response.get('albums', [])

    # 리스트 컴프리헨션을 사용하여 특정 앨범의 ID 찾기
    album_id = next((album['id'] for album in albums if album['title'] == album_title), None)

    # """OCR 파이프라인 실행 함수"""
    photo_urls = get_photo_urls(album_id)
    for idx, url in enumerate(photo_urls, start=1):
        if idx == 3:
            print(f'Processing image {idx}/{len(photo_urls)}')
            document = annotate_image(url)
            ocr_text = convert_ocr_to_text(document)
            file_name = apkg_title + str(idx) + '.apkg'
            write_to_file(file_name, generate_anki_deck(ocr_text, deck_title))


def generate_anki_deck(ocr_texts, name):
    """Anki 덱을 생성하는 함수"""
    # 덱 지정
    my_deck = genanki.Deck(12347778, name)
    # 모델 지정
    anki_my_model = genanki.Model(
        109123411,
        '객관식 정답과 해설 모델',
        fields=[
            {'name': 'question_no'},
            {'name': 'question'},
            {'name': 'choices'},
            {'name': 'answer'}
        ],
        templates=[
            {'name': 'Card {{question_no}}',
             'qfmt': '{{question_no}}. {{question}}<br><br>{{choices}}',
             'afmt': '{{FrontSide}}<hr id="answer">{{answer}}'}
        ]
    )
    response_open = client.chat.completions.create(
        messages=[
            {"role": "user", "content": "I have the following text"},
            {"role": "user", "content": ocr_texts},
            {"role": "user", "content":
                "이게 문제목록이야. 두자리 숫자로 시작되는 문제가 있고 question_no 필드로 문자형태로 되어있어. 앞 문제번호와 그다음 문제번호는 1씩 차이나. "
                "예를 들어 question_no가 '07'이면 그 앞문제 번호는 '06'이 되 그 뒤문제 번호는 '08이야. '594'는 '68'번 문제야"
                "문제는 4~6개씩 있어. 문제번호가 없는 경우 앞뒤 상황으로 유추해줘. 문제는 빅데이터분석기사의 내용에 맞는건지 확인해줘"
                "①과 같은 형태로 된건 보기야. 문제번호인 question_no는 문자열로 바뀐형태야."
                "배열 형태인 choices 가 있고 choices 는 보기4개로 이뤄진 list. 각 list 는 숫자.보기 내용 텍스트로 되어있어."
                "각 choices 에 ①과 같은 숫자기호가 없는 choices 들은 유추해서 넣어. "
                "숫자기호는 1,2,3,4로 대체해서 넣어. 모든 숫자기호는 ①과 같은 기호가 아닌 숫자로 대체해"
                "choices 의 choice 는 각각 한줄이 되게 띄어쓰기를 붙여줘. choices 는 4개의 보기로 이뤄진 list."
                "choices 의  첫번째 보기는 1.로 시작하고, 두번째 보기는 2.로 시작하고 세번째 보기는 3.으로 시작하고 네번째 보기는 4.으로 시작해."
                "보기내용이 없으면 빅데이터분석기사의 내용으로 문제에 맞는 보기를 유추해서  < 는 &lt;로 바꾸고 > 는 &gt;로 바꿔서 넣어줘"
                "문제와 문제사이가 보기니까 다음문제 앞에 ①과 같은 보기영역을 찾아서 4로 나누어 잘 매핑해줘."
                "다음의 의사 결정 나무에서 x값을 구하시오.와 같은 문제의 경우 51번 문제 윗줄들 보기 4줄만 처리해줘. 문제와 보기사이의 내용들은 문제내용으로 문제쪽으로 붙여줘."
                " < 또는 > 같은건 escape 처리해줘"
                "모든 문제, 보기값들, 답값은 escape 처리해줘. "
                "< 는 &lt;로 바꾸고 > 는 &gt;로 바꿔줘. 예를 들어 1. x < 10는 1. x &lt; 10으로 바꿔줘. 꼭 html escape 처리를 해줘"
                "questions 는 question_no, question, choices, answer,explanation 5개 필드를 가진 object 이고 json."
                "만약 json 되기 어려우면 question_no는 유추한 그대로, question 이 없으면 문제깨짐 이라고 넣고 json 맞춰줘."
                "answer 는 question_no에 맞는 정답번호를 찾아 answer_list 매핑해준 뒤에 추가로 한칸 띄우고 정답에 맞는 보기내용도 같이 매핑해줘"
                "explanation 필드는 문제와 보기에 대한 해설도 빅데이터분석기사내용을 토대로 자세히 넣어줘"
                + str(anki_my_model) + "의 구조에 맞게 "
                                       "각각 question_no, question, choices, answer,explanation 5개 필드를 가진 object 되어있어"
                                       "코드블럭표시하는 ```json ```제거하고 줘. 설명도 제거하고 questions 필드를 가진 json 만들어줘. 꼭  questions "
                                       "필드를 가진 object 여야해. 중간"
                                       "생략없이 줘"}
        ],
        model=open_ai_model
    )
    result_data = response_open.choices[0].message.content
    result_data = result_data.replace('```json', '').replace('```', '')

    try:
        json_object = json.loads(result_data)
        print('json.loads')
        print(json_object)
    except json.JSONDecodeError:
        print('JSON 형식이 아닙니다.')
        print(result_data)
        json_object = json.load(result_data)
        print('json.load')
        print(result_data)

    try:
        json_object.get('questions')
    except AttributeError:
        print('questions 속성이 없습니다.')
        print(result_data)
        json_object.append({'questions': json_object})

    for question_data in json_object.get('questions', []):
        if question_data['choices'] != '':
            question = html.escape(question_data['question'])
            choices = html.escape(''.join(question_data['choices']))
            answer = f"정답: {question_data['answer']} 해설: {html.escape(question_data['explanation'])}"
            note = genanki.Note(
                model=anki_my_model,
                fields=[question_data['question_no'], question, choices, answer]
            )
            my_deck.add_note(note)
    return my_deck


def write_to_file(file_name, my_deck):
    """Anki 패키지 파일을 생성하는 함수"""
    genanki.Package(my_deck).write_to_file(file_name)
    print(f'{file_name} 파일이 생성되었습니다.')


# 코드 실행############################################################################################################
if __name__ == "__main__":
    # 앨범 지정
    album_name = '기출'
    # 덱 버전 지정
    deck_version = 'VER1'
    # 덱 이름 지정
    deck_name = album_name + '_' + deck_version + '_DECK'
    # 패키지 이름 지정
    apkg_name = deck_name
    run_ocr_pipeline(album_name, deck_name, apkg_name)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=321040457013-9c5098pkpmsdma3c2psajfong2kdv1kq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8088%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fphotoslibrary.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-vision&state=YcpVGgu6kcBj68nko7CE93UkWDg3TI&access_type=offline
Processing image 3/17
try JSON Object
{'questions': [{'question_no': '16', 'question': '다음 중 데이터 분석 조직에 대한 설명으로 옳지 않은 것은?', 'choices': ['1. 기능형은 특정 현업 부서에 국한된 협소한 분석을 수행할 가능성이 높다.', '2. 집중형은 전사 분석 업무를 별도의 전담 조직에서 수행하므로 중복되지 않는다.', '3. 분산형은 분석 전문 인력을 현업 부서에 배치하여 분석 업무를 신속하게 수행한다.', '4. 조직 구조는 집중형, 기능형, 분산형으로 구분할 수 있으며, 기능형은 DSCoE 조직이 없다.'], 'answer': '4', 'explanation': '기능형은 DSCoE 조직이 존재하므로, 옳지 않은 설명이다.'}, {'question_no': '17', 'question': '다음 중 데이터를 추출하여 저장하는 기술로 옳은 것은?', 'choices': ['1. ETL', '2. OLAP', '3. Hadoop', '4. Data Mart'], 'answer': '1', 'explanation': ''}